# decorator
- after learning closure. We essentially modified our add function by wrapping it inside another function that added some more funtionality to it.
- We also say that we decorated our add function with the function couter.
- counter was a decorator.
### In general a decorator function:
    - takes a function as an argument
    - returns a closure.
    - the closure usually accpets any combination if parameters.
    - run some code in the inner function (closure)
    - the closure function calls the original function using the arguments passed to the closure.
    - returns whatever is returns by the function call.

In [24]:
def counter(fn):
    count = 0

    def inner(*args,**kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args,**kwargs)
    return inner

def add(a,b=2):
    return a+b

add = counter(add)
add(6)

Function add was called 1 times


8

In [25]:
@counter
def mul(a,b=2):
    """returns the product of 2 and a integer"""
    print(mul.__name__) # inner -> not mult->mult name changed when we decorated it.
    return a*b

mul(3)

Function mul was called 1 times
inner


6

In [26]:
import inspect
print(help(mul))# its not mult -> so we lost the doc, signature.
inspect.signature(mul)

Help on function inner in module __main__:

inner(*args, **kwargs)

None


<Signature (*args, **kwargs)>

# to fix it.
- the functools module has a wraps function that we can use to tfix the metadata of our inner function in our decorator.
- wraps function is itself a decorator.

In [27]:
from functools import wraps

def counter(fn):
    count = 0

    @wraps(fn) # using decorator # we dont have to use @waros, but it will make debugging easier. # can be used closure too. ( done in before return inner.)
    def inner(*args,**kwargs):
        nonlocal count
        count += 1
        print(f'Function {fn.__name__} was called {count} times')
        return fn(*args,**kwargs)
    # inner.__name__ = fn.__name__ # overridding the function
    # inner.__doc__ = fn.__doc__ # overridding the function
    # the function overring dont solve the function signature so,
    # inner = wraps(fn)(inner) # using closure
    return inner

def add(a,b=2):
    return a+b


@counter
def mul(a,b=2):
    """returns the product of 2 and a integer"""
    return a*b
print(mul(4))
print(help(mul))
inspect.signature(mul)

Function mul was called 1 times
8
Help on function mul in module __main__:

mul(a, b=2)
    returns the product of 2 and a integer

None


<Signature (a, b=2)>

# decorator Application @timed

In [115]:
def timed(fn,n=10):
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args,**kwargs):
        total_elapsed = 0
        total_count = 0

        for i in range(n):
            start = perf_counter()
            result = fn(*args,**kwargs)
            end = perf_counter()
            elapsed = end -start

            total_elapsed +=elapsed
            total_count +=1

        args_ = [str(a) for a in args]
        kwargs_ = [f'{k}={v}' for k,v in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)
        total_avg = total_elapsed/total_count

        print(f'{fn.__name__} ({args_str}) took {total_avg:.6f}s to run')
        return result
    return inner

# @timed
def fibo_recursive(n):
    if n<=2:
        return 1
    else:
        return fibo_recursive(n-1)+fibo_recursive(n-2)

@timed
def calc_fibo_recursive(n):
    return fibo_recursive(n)

@timed
def fib_loop(n):
    f1,f2 = 1,1
    for i in range(3,n+1):
        f1,f2 = f2,f1+f2
    return f2

# python reduce is slower then for loop.
# if python have something, means not need to use it.

In [77]:
print(calc_fibo_recursive(32))
print(calc_fibo_recursive(33))

calc_fibo_recursive (32) took 0.687525s to run
2178309
calc_fibo_recursive (33) took 1.081854s to run
3524578
calc_fibo_recursive (34) took 1.684864s to run
5702887


In [93]:
fib_loop(32) # faster then recursion

fib_loop (32) took 0.000005s to run


2178309

In [95]:
# the problem here is, we cannot say the no.of loop in the decorator. it is fixed to 10.
# the soln is parameter decorator.
# Either we use closure like:
fib_loop = timed(fib_loop,15) # we cannot use @timed
fib_loop(32)

fib_loop (32) took 0.000003s to run
fib_loop (32) took 0.000002s to run
fib_loop (32) took 0.000017s to run
fib_loop (32) took 0.000002s to run
fib_loop (32) took 0.000003s to run
fib_loop (32) took 0.000005s to run
fib_loop (32) took 0.000004s to run
fib_loop (32) took 0.000003s to run
fib_loop (32) took 0.000004s to run
fib_loop (32) took 0.000004s to run
fib_loop (32) took 0.000004s to run
fib_loop (32) took 0.000004s to run
fib_loop (32) took 0.000004s to run
fib_loop (32) took 0.000004s to run
fib_loop (32) took 0.000004s to run
fib_loop (32) took 0.000155s to run


2178309

# decorator Application (Logger,Stacked Decorators)
- Nested decorator

In [123]:
def logged(fn):
    from functools import wraps
    from datetime import datetime

    @wraps(fn)
    def inner(*args,**kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args,**kwargs)

        print(f'{run_dt} : called {fn.__name__}')
        return result
    return inner

def timed(fn):
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args,**kwargs):
      
        start = perf_counter()
        result = fn(*args,**kwargs)
        end = perf_counter()
        elapsed = end -start

        args_ = [str(a) for a in args]
        kwargs_ = [f'{k}={v}' for k,v in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)

        print(f'{fn.__name__} ({args_str}) took {elapsed:.6f}s to run')
        return result
    return inner


@logged
@timed
def fact(n):
    from functools import reduce
    return reduce(lambda x,y:x*y,range(1,n+1))

# using closure
def fact1(n):
    from functools import reduce
    return reduce(lambda x,y:x*y,range(1,n+1))
fact1 = logged(timed(fact1))
fact1(3)

fact1 (3) took 0.000008s to run
2021-06-02 12:06:26.042689+00:00 : called fact1


6

In [124]:
fact(3)

fact (3) took 0.000012s to run
2021-06-02 12:06:28.968641+00:00 : called fact


6

# Decorator Application (Memoization)
- It allows us to build cache of the function values of the returned values, based on the input parameters.
- Decrease the computational time.

In [142]:
def calc_fib(n):
    print('calculating fib',n)
    return 1 if n<3 else calc_fib(n-1)+calc_fib(n-2)

calc_fib(10)

calculating fib 10
calculating fib 9
calculating fib 8
calculating fib 7
calculating fib 6
calculating fib 5
calculating fib 4
calculating fib 3
calculating fib 2
calculating fib 1
calculating fib 2
calculating fib 3
calculating fib 2
calculating fib 1
calculating fib 4
calculating fib 3
calculating fib 2
calculating fib 1
calculating fib 2
calculating fib 5
calculating fib 4
calculating fib 3
calculating fib 2
calculating fib 1
calculating fib 2
calculating fib 3
calculating fib 2
calculating fib 1
calculating fib 6
calculating fib 5
calculating fib 4
calculating fib 3
calculating fib 2
calculating fib 1
calculating fib 2
calculating fib 3
calculating fib 2
calculating fib 1
calculating fib 4
calculating fib 3
calculating fib 2
calculating fib 1
calculating fib 2
calculating fib 7
calculating fib 6
calculating fib 5
calculating fib 4
calculating fib 3
calculating fib 2
calculating fib 1
calculating fib 2
calculating fib 3
calculating fib 2
calculating fib 1
calculating fib 4
calculati

55

In [146]:
# lets first use class to cache or store the calculated value in dict.
class Fib:
    def __init__(self):
        self.cache = {1:1,2:1}

    def fib(self,n):
        if n not in self.cache:
            print('Calculating fib',n)
            self.cache[n] = self.fib(n-1)+self.fib(n-2)
        return self.cache[n]

    def __call__(self,n):
        return self.fib(n)

fi = Fib()
fi(10)
# but the prob is is we create another object then it will calculate again.
fj = Fib()
fj(10)

Calculating fib 10
Calculating fib 9
Calculating fib 8
Calculating fib 7
Calculating fib 6
Calculating fib 5
Calculating fib 4
Calculating fib 3
Calculating fib 10
Calculating fib 9
Calculating fib 8
Calculating fib 7
Calculating fib 6
Calculating fib 5
Calculating fib 4
Calculating fib 3


55

In [147]:
# lets use closure to cache the calcualted value
def fib():
    cache = {1:1,2:1}

    def inner(n):
        if n not in cache:
            print('calculating fib ',n)
            cache[n] = inner(n-1)+inner(n-2)
        return cache[n]
    return inner

f = fib()
f(10)
# but the prob is is we create another object then it will calculate again.
ff = fib()
ff(10)

calculating fib  10
calculating fib  9
calculating fib  8
calculating fib  7
calculating fib  6
calculating fib  5
calculating fib  4
calculating fib  3
calculating fib  10
calculating fib  9
calculating fib  8
calculating fib  7
calculating fib  6
calculating fib  5
calculating fib  4
calculating fib  3


55

In [161]:
# lets do it with decorator
def memoize(fn):
    cache = dict() # no limit in cache. we can say 1000 records in cache and then remove least recently used. but hehe,  its built-in.

    def inner(n): # but the prob is we dont handle *args and **kwargs. here args can be used.
        if n not in cache:
            cache[n] = fn(n) # cache[args] , a tuple can be key to a dictionary, but it depends on what is in that tuple. it is more reliable if we hash it.
        return cache[n]
    return inner

@memoize
def calc_fib(n):
    print('calculating fib',n)
    return 1 if n<3 else calc_fib(n-1)+calc_fib(n-2)

print(calc_fib(10))
print(calc_fib(9)) # here same cache is used.
print(calc_fib(11))

@memoize
def calc_fact(n):
    print('calculating fact ',n)
    return 1 if n<2 else calc_fact(n-1)*n

print(calc_fact(7))
print(calc_fact(6))
print(calc_fact(3))

calculating fib 10
calculating fib 9
calculating fib 8
calculating fib 7
calculating fib 6
calculating fib 5
calculating fib 4
calculating fib 3
calculating fib 2
calculating fib 1
55
34
calculating fib 11
89
calculating fact  7
calculating fact  6
calculating fact  5
calculating fact  4
calculating fact  3
calculating fact  2
calculating fact  1
5040
720
6


In [172]:
# let use lru_cache
from functools import lru_cache

@lru_cache(maxsize=8) # default size is 128 -> 2^size
def fib(n):
    print('Calculating fib',n)
    return 1 if n<3 else fib(n-1)+fib(n-2)

print(fib(8))
print(fib(4))
print(fib(22))
print(fib(8))

Calculating fib 8
Calculating fib 7
Calculating fib 6
Calculating fib 5
Calculating fib 4
Calculating fib 3
Calculating fib 2
Calculating fib 1
21
3
Calculating fib 22
Calculating fib 21
Calculating fib 20
Calculating fib 19
Calculating fib 18
Calculating fib 17
Calculating fib 16
Calculating fib 15
Calculating fib 14
Calculating fib 13
Calculating fib 12
Calculating fib 11
Calculating fib 10
Calculating fib 9
17711
Calculating fib 8
Calculating fib 7
Calculating fib 6
Calculating fib 5
Calculating fib 4
Calculating fib 3
Calculating fib 2
Calculating fib 1
21


# Decorators with parameter

In [193]:
def timed(n=10):
    def outer(fn):
        from time import perf_counter
        from functools import wraps

        @wraps(fn)
        def inner(*args,**kwargs):
            total_elapsed,total_count = 0,0

            for i in range(n):
                start = perf_counter()
                result = fn(*args,**kwargs)
                end = perf_counter()
                elapsed = end -start

                total_elapsed +=elapsed
                total_count +=1

            total_avg = total_elapsed/total_count
            print(f'{fn.__name__} took {total_avg:.6f}s to run')
            return result
        return inner
    return outer
 
def fibo_recursive(n):
    return 1 if n<3 else fibo_recursive(n-1)+fibo_recursive(n-2)

@timed(4) # @timed is a decorator factory.
def calc_fibo_recursive(n): 
    return fibo_recursive(n)


In [195]:
calc_fibo_recursive(20)

calc_fibo_recursive took 0.003272s to run


6765

## Decorator Factories
    - The timed function is not itself a Decorator. Instead it returns a Decorator when called.
    - any argument passed to timed eg: 'n', can be referenced (as free variable) inside out decorator.
- ### we call this timed function a decorator factory function because when we call it, it returns a decorator.

In [19]:
import functools

user = {'username':'rabin','access_level':'admin'}

def make_secure(access_level):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args,**kwargs):
            return func(*args,**kwargs) if user['access_level']==access_level else f"No {access_level} permission for {user['username']}"
        return wrapper
    return decorator

@make_secure('admin')
def get_admin_password():
    return "admin: 1234"

@make_secure('user')
def get_dashboard_password():
    return "dashboard: password"


print(get_admin_password())
print(get_dashboard_password())

admin: 1234
No user permission for rabin


# Decorator Application - Single Dispatch Generic Functions.
- usecase:
    - format html data

In [39]:
from html import escape
from decimal import Decimal

def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return f'{a} <i> {str(hex(a))} </i>'

def html_real(a):
    return '0:.2f'.format(round(a,2))

def html_str(s):
    return html_escape(s).replace('\n','<br/>\n')

def html_list(l):
    items = (f'<li>{htmlize(item)}</li>' for item in l)
    return '<ul>\n'+'\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = (f'<li>{k}={htmlize(v)}</li>' for k,v in d.items())
    return '<ul>\n'+'\n'.join(items)+'\n</ul>'

def html_set(arg):
    return html_list(arg)

In [37]:
def htmlize(arg):
    registry={
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_real,
        str: html_str,
        list: html_list,
        tuple: html_list,
        set: html_set,
        dict: 
        html_dict
    }
    fn = registry.get(type(arg),registry[object])
    return fn(arg)

In [38]:
print(html_int(255))
print(htmlize(["""this is the 
multiline strong""",{1,2,3},100]))

255 <i> 0xff </i>
<ul>
<li>this is the <br/>
multiline strong</li>
<li><ul>
<li>1 <i> 0x1 </i></li>
<li>2 <i> 0x2 </i></li>
<li>3 <i> 0x3 </i></li>
</ul></li>
<li>100 <i> 0x64 </i></li>
</ul>


In [30]:
htmlize(100)

'100 <i> 0x64 </i>'

## we have hard coded our registry.
## It is done, but what if we have add new custom type to that registry dictionary.

In [91]:
def singledispatch(fn):
    registry = {}

    registry[object]= fn
    def decorator(arg):
        return registry.get(type(arg),registry[object])(arg)

    def register(type_): # parameterized decorator inside decorator
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    def dispatch(type_):
        return registry.get(type_,registry[object])

    decorator.register = register
    decorator.dispatch = dispatch
    # add attribute dynamically at run-time.
    # monkey patching.
    return decorator

In [92]:
@singledispatch
def htmlize(a):
    return escape(str(a))

htmlize(100)

'100'

In [93]:
@htmlize.register(int)
def html_int(a):
    return f'{a} <i> {str(hex(a))} </i>'

In [94]:
htmlize(100)

'100 <i> 0x64 </i>'

In [95]:
@htmlize.register(set)
@htmlize.register(tuple)
@htmlize.register(list)
def html_sequence(l):
    items = (f'<li>{htmlize(item)}</li>' for item in l)
    return '<ul>\n'+'\n'.join(items) + '\n</ul>'

In [103]:
print(htmlize({1,2,3}))

<ul>
<li>1 <i> 0x1 </i></li>
<li>2 <i> 0x2 </i></li>
<li>3 <i> 0x3 </i></li>
</ul>


In [107]:
from numbers import Integral
@htmlize.register(int)
@htmlize.register(Integral)
def html_int(a):
    return f'{a} <i> {str(hex(a))} </i>'

In [108]:
print(htmlize(10))

10 <i> 0xa </i>


In [100]:
htmlize.dispatch(Integral)

<function __main__.html_integral(i)>

In [116]:
## Here the problem is that:
# 10 is int as well as integer. In the decorator, we use type() to check and return the function that matches the type of argument we passed.

In [119]:
# Same as in sequence.
from collections.abc import Sequence
print(isinstance([1,2],Sequence))
# BUT
print(type([1,2]) is Sequence)
# So, we have to use, isinstance in the decorator.

True
False


## Fixing the type to isinstance using built-in singledispatch

In [1]:
from functools import singledispatch
from numbers import Integral
from collections.abc import Sequence
from html import escape

@singledispatch
def htmlize(a): # will be default function.
    return escape(str(a))

htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>})

In [2]:
@htmlize.register(Integral)
def htmlize_integral_number(a):
    return f'{a} <i>{str(hex(a))} </i>'

In [3]:
htmlize(10)
# so here
print(type(10))
print(isinstance(10,int))
print(isinstance(10,Integral))
# same with the boolean
print(isinstance(True,Integral))

<class 'int'>
True
True
True


In [4]:
@htmlize.register(Sequence)
def html_sequence(l):
    items = (f'<li>{htmlize(item)}</li>' for item in l)
    return '<ul>\n'+'\n'.join(items) + '\n</ul>'

In [5]:
print(htmlize([1,2,3]))

print(type([1,2]))
print(isinstance([1,2],Sequence))
print(isinstance((1,2),Sequence))

print(isinstance({1,2},Sequence)) # set

<ul>
<li>1 <i>0x1 </i></li>
<li>2 <i>0x2 </i></li>
<li>3 <i>0x3 </i></li>
</ul>
<class 'list'>
True
True
False


In [17]:
# if we want to do specific for tuples
@htmlize.register(tuple)
def html_tuple(t):
    items = (escape(str(item)) for item in t)
    return f'({", ".join(items)})'

In [20]:
htmlize((1,2))

'(1, 2)'

In [14]:
print('"python": is sequence->',isinstance('python',Sequence))
# htmlize('python') # Max exceed error.
# we dont want to handle string
@htmlize.register(str)
def html_str(s):
    return escape(s).replace('\n','<br/>\n')

"python": is sequence-> True


In [15]:
htmlize('python 1<10')

'python 1&lt;10'

## The best approach is to use function name
- def _():
- So, that we cannot call the function directly.
- We should use htmlizer

In [21]:
from functools import singledispatch
from numbers import Integral
from collections.abc import Sequence
from html import escape

@singledispatch
def htmlize(a): # will be default function.
    return escape(str(a))

htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>})

In [35]:
@htmlize.register(Integral)
def _(a): return f'{a} <i>{str(hex(a))} </i>'

@htmlize.register(Sequence)
def _(l): # look for function lable pointing-> memory address as refrence. all function have diff address.
    items = (f'<li>{htmlize(item)}</li>' for item in l)
    return '<ul>\n'+'\n'.join(items) + '\n</ul>'

@htmlize.register(tuple)
def _(t):
    items = (escape(str(item)) for item in t)
    return f'({", ".join(items)})'

@htmlize.register(set)
def _(s):
    items = (escape(str(item)) for item in s)
    return f'({"- ".join(items)})'

@htmlize.register(str)
def _(s): return escape(s).replace('\n','<br/>\n')

In [34]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__._(a)>,
              collections.abc.Sequence: <function __main__._(l)>,
              tuple: <function __main__._(t)>,
              str: <function __main__._(s)>,
              set: <function __main__._(s)>})

In [33]:
print(htmlize([1,2,3]))
print(htmlize({1,8,9}))

<ul>
<li>1 <i>0x1 </i></li>
<li>2 <i>0x2 </i></li>
<li>3 <i>0x3 </i></li>
</ul>
(8- 1- 9)
